In [1]:
import os
import shutil
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yaml


In [11]:
def create_yolo_yaml_config(yaml_filepath, dataset_path, dataset_labels):

    data = {'path':dataset_path,
            'train': os.path.join('images', 'train'),
            'val': os.path.join('images', 'validation'),
            'names':{i:label for i, label in enumerate(dataset_labels)}
            }

    # Save the changes to the file
    with open(yaml_filepath, 'w') as fp:
        yaml.dump(data, fp, sort_keys=False)

# ******************************************************************************************

In [12]:
def get_class_id(classes_file, class_names):
    id_name_dict = {}
    with open(classes_file, 'r') as f:
        for line in f:
            id, label = line.split(',')
            label = label.strip()
            #print(label)
            if label in class_names:
                print(label)
                id_name_dict[label] = id
        
    return id_name_dict
#*******************************************************************************



In [ ]:
"""import shutil

for f in os.listdir('/kaggle/working/'):
    shutil.rmtree(f)"""

In [5]:
!wget https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv

--2023-04-14 20:53:02--  https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.159.128, 142.251.161.128, 74.125.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.159.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2258447590 (2.1G) [text/csv]
Saving to: ‘oidv6-train-annotations-bbox.csv’

oidv6-train-annotat 100%[===================>]   2.10G   151MB/s    in 14s     

2023-04-14 20:53:16 (156 MB/s) - ‘oidv6-train-annotations-bbox.csv’ saved [2258447590/2258447590]



In [6]:
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv

--2023-04-14 20:53:17--  https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 173.194.198.128, 173.194.74.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25105048 (24M) [text/csv]
Saving to: ‘validation-annotations-bbox.csv’

validation-annotati 100%[===================>]  23.94M   115MB/s    in 0.2s    

2023-04-14 20:53:17 (115 MB/s) - ‘validation-annotations-bbox.csv’ saved [25105048/25105048]



In [7]:
!wget https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv

--2023-04-14 20:53:18--  https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 173.194.198.128, 173.194.74.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77484237 (74M) [text/csv]
Saving to: ‘test-annotations-bbox.csv’

test-annotations-bb 100%[===================>]  73.89M   137MB/s    in 0.5s    

2023-04-14 20:53:19 (137 MB/s) - ‘test-annotations-bbox.csv’ saved [77484237/77484237]



In [8]:
!git clone https://github.com/mohamedamine99/YOLOv8-custom-object-detection

Cloning into 'YOLOv8-custom-object-detection'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 57 (delta 22), reused 38 (delta 10), pack-reused 7
Receiving objects: 100% (57/57), 80.79 MiB | 34.31 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [15]:
names = ['Alpaca']
dataset_path = os.path.abspath(os.path.join('.', 'data')) # recommended to use absolute path 
yaml_filepath = os.path.join('.', 'config.yaml')
create_yolo_yaml_config(yaml_filepath, dataset_path, names)

In [17]:
class_ids = get_class_id('/kaggle/working/YOLOv8-custom-object-detection/class-descriptions-boxable.csv'
                         ,names)
print(class_ids)
print(names)

Alpaca
{'Alpaca': '/m/0pcr'}
['Alpaca']


In [18]:
# Create a list of annotated images to be downloaded by the downloader script download.py
# the list is a text file in the following format : $SPLIT/$IMAGE_ID
# example : 
# train/f9e0434389a1d4dd
# train/1a007563ebc18664

#import os

#alpaca_id = '/m/0pcr'

#names = ['alpaca']

train_bboxes_filename = os.path.join('.', 'oidv6-train-annotations-bbox.csv')
validation_bboxes_filename = os.path.join('.', 'validation-annotations-bbox.csv')
test_bboxes_filename = os.path.join('.', 'test-annotations-bbox.csv')

image_list_file_path = os.path.join('.', 'image_list_file.txt')

image_list_file_list = []
for j, filename in enumerate([train_bboxes_filename, validation_bboxes_filename, test_bboxes_filename]):
    print(filename)
    with open(filename, 'r') as f:
        line = f.readline()
        while len(line) != 0:
            id, _, class_name, _, x1, x2, y1, y2, _, _, _, _, _ = line.split(',')[:13]
            if class_name in list(class_ids.values()) and id not in image_list_file_list:
                image_list_file_list.append(id)
                with open(image_list_file_path, 'a') as fw:
                    fw.write('{}/{}\n'.format(['train', 'validation', 'test'][j], id))
            line = f.readline()

        f.close()

./oidv6-train-annotations-bbox.csv
./validation-annotations-bbox.csv
./test-annotations-bbox.csv


In [19]:
!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

--2023-04-14 21:03:43--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘downloader.py’

downloader.py       100%[===================>]   4.14K  --.-KB/s    in 0s      

2023-04-14 21:03:44 (31.4 MB/s) - ‘downloader.py’ saved [4244/4244]



In [20]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 69.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.114 which is incompatible.


In [21]:
DATA_ALL_DIR = os.path.join('.', 'data_all') # all downloaded data selected from the list
DATA_OUT_DIR = os.path.join('.', 'data') # data reorganized in YOLO format
os.makedirs(DATA_ALL_DIR)
os.makedirs(DATA_OUT_DIR)

In [22]:
!python downloader.py ./image_list_file.txt --download_folder=/kaggle/working/data_all --num_processes=5

In [23]:
# Create a dataset in the yolo format with two main directories : images and labels
# each containing train and validation data 


for dir_ in ['images', 'labels']:
    for set_ in ['train', 'validation', 'test']:
        new_dir = os.path.join(DATA_OUT_DIR, dir_, set_)
        if os.path.exists(new_dir):
            shutil.rmtree(new_dir)
        os.makedirs(new_dir)


In [24]:
#alpaca_id = '/m/0pcr'

#train_bboxes_filename = os.path.join('.', 'oidv6-train-annotations-bbox.csv')
#validation_bboxes_filename = os.path.join('.', 'validation-annotations-bbox.csv')
#test_bboxes_filename = os.path.join('.', 'test-annotations-bbox.csv')


for j, filename in enumerate([train_bboxes_filename, validation_bboxes_filename, test_bboxes_filename]):
    set_ = ['train', 'validation', 'test'][j]
    print(filename)
    with open(filename, 'r') as f:
        line = f.readline()
        while len(line) != 0:
            id, _, class_name, _, x1, x2, y1, y2, _, _, _, _, _ = line.split(',')[:13]
            if class_name in list(class_ids.values()):

                if not os.path.exists(os.path.join(DATA_OUT_DIR, 'images', set_, '{}.jpg'.format(id))):

                    shutil.copy(os.path.join(DATA_ALL_DIR, '{}.jpg'.format(id)),
                                os.path.join(DATA_OUT_DIR, 'images', set_, '{}.jpg'.format(id)))
                    
                with open(os.path.join(DATA_OUT_DIR, 'labels', set_, '{}.txt'.format(id)), 'a') as f_ann:
                    # class_id, xc, yx, w, h
                    x1, x2, y1, y2 = [float(j) for j in [x1, x2, y1, y2]]
                    xc = (x1 + x2) / 2
                    yc = (y1 + y2) / 2
                    w = x2 - x1
                    h = y2 - y1

                    f_ann.write('0 {} {} {} {}\n'.format(xc, yc, w, h))
                    f_ann.close()

            line = f.readline()

./oidv6-train-annotations-bbox.csv
./validation-annotations-bbox.csv
./test-annotations-bbox.csv


In [25]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.77 🚀 Python-3.7.12 torch-1.13.0 CUDA:0 (Tesla T4, 15110MiB)
Setup complete ✅ (2 CPUs, 15.6 GB RAM, 4555.2/8062.4 GB disk)


In [29]:
from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.pt")  # build a new model from scratch

# Use the model
results = model.train(data='/kaggle/working/config.yaml', 
                      epochs=150)  # train the model


Ultralytics YOLOv8.0.77 🚀 Python-3.7.12 torch-1.13.0 CUDA:0 (Tesla T4, 15110MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/config.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simp

In [30]:
!zip -r train_results.zip /kaggle/working/runs

updating: kaggle/working/runs/ (stored 0%)
updating: kaggle/working/runs/detect/ (stored 0%)
updating: kaggle/working/runs/detect/train/ (stored 0%)
updating: kaggle/working/runs/detect/train/labels.jpg (deflated 24%)
updating: kaggle/working/runs/detect/train/P_curve.png (deflated 16%)
updating: kaggle/working/runs/detect/train/val_batch0_labels.jpg (deflated 11%)
updating: kaggle/working/runs/detect/train/R_curve.png (deflated 17%)
updating: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 2%)
updating: kaggle/working/runs/detect/train/confusion_matrix.png (deflated 39%)
updating: kaggle/working/runs/detect/train/train_batch1.jpg (deflated 3%)
updating: kaggle/working/runs/detect/train/train_batch2.jpg (deflated 3%)
updating: kaggle/working/runs/detect/train/results.png (deflated 7%)
updating: kaggle/working/runs/detect/train/val_batch0_pred.jpg (deflated 11%)
updating: kaggle/working/runs/detect/train/labels_correlogram.jpg (deflated 39%)
updating: kaggle/working/runs/det